In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_novice=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_novice.csv')
elos_avg=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_avg.csv')
elos_master=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_master.csv')
elos_grandmaster=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos_grandmaster.csv')
features_novice=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_novice.csv')
features_master=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_master.csv')
features_avg=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_avg.csv')
features_grandmaster=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features_grandmaster.csv')

In [4]:
X_train_mean_novice, X_test_mean_novice, Y_train_mean_novice, Y_test_mean_novice =ms.train_test_split(features_novice, elos_novice['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_novice, X_test_diff_novice, Y_train_diff_novice, Y_test_diff_novice =ms.train_test_split(features_novice, elos_novice['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_avg, X_test_mean_avg, Y_train_mean_avg, Y_test_mean_avg =ms.train_test_split(features_avg, elos_avg['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_avg, X_test_diff_avg, Y_train_diff_avg, Y_test_diff_avg =ms.train_test_split(features_avg, elos_avg['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_master, X_test_mean_master, Y_train_mean_master, Y_test_mean_master =ms.train_test_split(features_master, elos_master['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_master, X_test_diff_master, Y_train_diff_master, Y_test_diff_master =ms.train_test_split(features_master, elos_master['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_grandmaster, X_test_mean_grandmaster, Y_train_mean_grandmaster, Y_test_mean_grandmaster =ms.train_test_split(features_grandmaster, elos_grandmaster['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_grandmaster, X_test_diff_grandmaster, Y_train_diff_grandmaster, Y_test_diff_grandmaster =ms.train_test_split(features_grandmaster, elos_grandmaster['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_mean=RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=-1, oob_score=False, random_state=4,
           verbose=0, warm_start=False)

model_diff=RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=25, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=True, random_state=4,
           verbose=0, warm_start=False)

In [6]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [7]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для новичков

In [8]:
start1 = time.time()

rf1=model_mean.fit(X_train_mean_novice,Y_train_mean_novice)
rf_pred1=rf1.predict(X_test_mean_novice)
scores1=predictions_report(Y_test_mean_novice,rf_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 190.557
RMSE = 241.754
Correlation coefficient = 0.199
52.901 c


In [9]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [10]:
start2 = time.time()

rf2=model_diff.fit(X_train_diff_novice,Y_train_diff_novice)
rf_pred2=rf2.predict(X_test_mean_novice)
scores2=predictions_report(Y_test_diff_novice,rf_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 175.868
RMSE = 219.280
Correlation coefficient = 0.563
18.861 c


In [11]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [12]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [13]:
white_elos_novice = rf_pred1 + rf_pred2/2
black_elos_novice = rf_pred1-rf_pred2/2
pred_elos1=list(zip(white_elos_novice,black_elos_novice))
print(len(pred_elos1))
pred_elos1[:5]

1935


[(2066.3975, 2089.4275),
 (2090.3714999999997, 2101.4085),
 (1993.5665000000001, 2067.4835000000003),
 (2099.092, 2017.038),
 (2071.1730000000002, 1987.5020000000002)]

In [14]:
find_abs_error(pred_elos1,elos_novice),find_mean_error(pred_elos1,elos_novice),find_root_mean_squared_error(pred_elos1,elos_novice)

(901549.197, 232.958, 290.788)

In [15]:
inner_mae_novice=find_mean_error(pred_elos1,elos_novice)

In [16]:
errors =errors.append(pd.Series(['Слабые игроки','Random Forest',
                  find_abs_error(pred_elos1,elos_novice),
                  find_mean_error(pred_elos1,elos_novice),
                   find_root_mean_squared_error(pred_elos1,elos_novice) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475


In [17]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для среднего уровня

In [18]:
start3 = time.time()

rf3=model_mean.fit(X_train_mean_avg,Y_train_mean_avg)
rf_pred3=rf3.predict(X_test_mean_avg)
scores3=predictions_report(Y_test_mean_avg,rf_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 149.148
RMSE = 194.654
Correlation coefficient = 0.081
44.961 c


In [19]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по начинающим'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [20]:
start4 = time.time()

rf4=model_diff.fit(X_train_diff_avg,Y_train_diff_avg)
rf_pred4=rf4.predict(X_test_mean_avg)
scores4=predictions_report(Y_test_diff_avg,rf_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 163.047
RMSE = 204.078
Correlation coefficient = 0.587
20.562 c


In [21]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по средним игрокам'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [22]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [23]:
white_elos_avg = rf_pred3 + rf_pred3/2
black_elos_avg = rf_pred3-rf_pred4/2
pred_elos2=list(zip(white_elos_avg,black_elos_avg))
print(len(pred_elos2))
pred_elos2[:5]

2125


[(3351.4987499999997, 2303.274),
 (3406.87125, 2307.9605),
 (3409.50375, 2283.651),
 (3361.8900000000003, 2268.6895000000004),
 (3324.1987500000005, 2289.8450000000003)]

In [24]:
find_abs_error(pred_elos2,elos_avg),find_mean_error(pred_elos2,elos_avg),find_root_mean_squared_error(pred_elos2,elos_avg)

(2804367.089, 659.851, 830.491)

In [25]:
inner_mae_avg=find_mean_error(pred_elos2,elos_avg)

In [26]:
errors =errors.append(pd.Series(['Средние игроки','Random Forest',
                        find_abs_error(pred_elos2,elos_avg),
                        find_mean_error(pred_elos2,elos_avg),
                        find_root_mean_squared_error(pred_elos2,elos_avg)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475


### Подбор модели для мастеров

In [27]:
start5 = time.time()

rf5=model_mean.fit(X_train_mean_master,Y_train_mean_master)
rf_pred5=rf5.predict(X_test_mean_master)
scores5=predictions_report(Y_test_mean_master,rf_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 145.077
RMSE = 188.076
Correlation coefficient = 0.140
63.469 c


In [28]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по мастерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [29]:
start6 = time.time()

rf6=model_diff.fit(X_train_diff_master,Y_train_diff_master)
rf_pred6=rf6.predict(X_test_mean_master)
scores6=predictions_report(Y_test_diff_master,rf_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 142.044
RMSE = 175.876
Correlation coefficient = 0.517
35.512 c


In [30]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по мастерам'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [31]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [32]:
white_elos_master = rf_pred5 + rf_pred6/2
black_elos_master = rf_pred5-rf_pred6/2
pred_elos3=list(zip(white_elos_master,black_elos_master))
print(len(pred_elos3))
pred_elos3[:5]

2904


[(2438.8825, 2278.1575),
 (2270.1845000000003, 2358.8755),
 (2341.3895, 2349.2555),
 (2282.685, 2391.6200000000003),
 (2386.562, 2307.1679999999997)]

In [33]:
find_abs_error(pred_elos3,elos_master),find_mean_error(pred_elos3,elos_master),find_root_mean_squared_error(pred_elos3,elos_master)

(1007724.918, 173.506, 220.821)

In [34]:
inner_mae_master=find_mean_error(pred_elos3,elos_master)

In [35]:
errors =errors.append(pd.Series(['Мастера','Random Forest',
                                 find_abs_error(pred_elos3,elos_master),
                                 find_mean_error(pred_elos3,elos_master),
                                 find_root_mean_squared_error(pred_elos3,elos_master)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475


### Подбор модели для гроссмейстеров

In [36]:
start7 = time.time()

rf7=model_mean.fit(X_train_mean_grandmaster,Y_train_mean_grandmaster)
rf_pred7=rf7.predict(X_test_mean_grandmaster)
scores7=predictions_report(Y_test_mean_grandmaster,rf_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

MAE = 153.484
RMSE = 196.130
Correlation coefficient = 0.110
2.331 c


In [37]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по гроссмейстерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [38]:
start8 = time.time()

rf8=model_diff.fit(X_train_diff_grandmaster,Y_train_diff_grandmaster)
rf_pred8=rf8.predict(X_test_mean_grandmaster)
scores8=predictions_report(Y_test_diff_grandmaster,rf_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

MAE = 103.624
RMSE = 133.434
Correlation coefficient = 0.274
1.555 c


In [39]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по гроссмейстерам'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",185.454373,232.791414,0.272042,0.104,Обучение на среднем ЭЛО по начинающим
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",167.847740,213.396908,0.604164,0.117,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",148.774801,195.120413,0.195972,0.104,Обучение на среднем ЭЛО по средним игрокам
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",153.680899,194.776371,0.621906,0.105,Обучение на разнице в ЭЛО по средним игрокам
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",144.547821,186.916248,0.211849,0.113,Обучение на среднем ЭЛО по мастерам
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",137.465903,174.230219,0.542416,0.120,Обучение на разнице в ЭЛО по мастерам
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",132.400047,179.033987,0.462350,0.026,Обучение на среднем ЭЛО по гроссмейстерам
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",91.863665,120.573687,0.523059,0.030,Обучение на разнице в ЭЛО по гроссмейстерам
8,"AdaBoostRegressor(base_estimator=None, learnin...",233.165385,286.956260,0.098477,47.529,Обучение на среднем ЭЛО по начинающим
9,"AdaBoostRegressor(base_estimator=None, learnin...",180.602289,225.785633,0.520742,5.654,Обучение на разнице ЭЛО по начинающим


In [40]:
white_elos_grandmaster = rf_pred7 + rf_pred8/2
black_elos_grandmaster = rf_pred7-rf_pred8/2
pred_elos4=list(zip(white_elos_grandmaster,black_elos_grandmaster))
print(len(pred_elos4))
pred_elos4[:5]

315


[(2547.1095, 2555.2255),
 (2478.8565000000003, 2491.7785),
 (2440.4714999999997, 2447.8035),
 (2504.3795, 2464.7805),
 (2356.9995, 2369.7955)]

In [41]:
find_abs_error(pred_elos4,elos_grandmaster),find_mean_error(pred_elos4,elos_grandmaster),find_root_mean_squared_error(pred_elos4,elos_grandmaster)

(106558.281, 169.14, 222.769)

In [42]:
inner_mae_grandmaster=find_mean_error(pred_elos4,elos_grandmaster)

In [43]:
errors =errors.append(pd.Series(['Гроссместеры','Random Forest',find_abs_error(pred_elos4,elos_grandmaster),
                                 find_mean_error(pred_elos4,elos_grandmaster),
                                 find_root_mean_squared_error(pred_elos4,elos_grandmaster)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475


In [44]:
rf_mae=(inner_mae_grandmaster+inner_mae_master+inner_mae_avg+inner_mae_novice)/4
rf_mae

308.86375

In [45]:
rf_abs=(find_abs_error(pred_elos1,elos_novice)+
        find_abs_error(pred_elos2,elos_avg)+
        find_abs_error(pred_elos3,elos_master)+
        find_abs_error(pred_elos4,elos_grandmaster))/4
rf_abs

1205049.87125

In [46]:
rf_rmse=(find_root_mean_squared_error(pred_elos1,elos_novice)+
         find_root_mean_squared_error(pred_elos2,elos_avg)+
         find_root_mean_squared_error(pred_elos3,elos_master)+
         find_root_mean_squared_error(pred_elos4,elos_grandmaster))/4
rf_rmse

391.21725

In [47]:
errors = errors.append(pd.Series(['Общая ошибка','Random Forest',
                  rf_abs,
                  rf_mae,
                 rf_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,Слабые игроки,LinearRegression,2.636124e+06,204.41400,256.08000
1,Средние игроки,LinearRegression,2.422902e+06,171.06100,218.08000
2,Мастера,LinearRegression,3.121464e+06,161.26600,206.22300
3,Гроссмейстеры,LinearRegression,2.933576e+05,140.09400,188.90800
4,Общая ошибка,LinearRegression,2.118462e+06,169.20875,217.32275
5,Слабые игроки,AdaBoost,1.053871e+06,272.31800,339.77300
6,Средние игроки,AdaBoost,3.092973e+06,727.75800,912.93700
7,Мастера,AdaBoost,1.130376e+06,194.62400,246.27300
8,Гроссместеры,AdaBoost,1.381863e+05,219.34300,297.87600
9,Общая ошибка,AdaBoost,1.353852e+06,353.51075,449.21475
